In [1]:
from binascii import b2a_hex
import os 
import sys
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter,PDFPageAggregator
from pdfminer.layout import LAParams, LTImage,LTFigure
from pdfminer.pdfpage import PDFPage,PDFParser,PDFDocument
from cStringIO import StringIO

def find_images_in_thing(outer_layout,i):
    for thing in outer_layout:
        if isinstance(thing, LTImage):
            save_image(thing,i)

In [29]:
home = 'C:/pj/Photo/'
from os import listdir
from os.path import isfile, join
onlyfiles = [f for f in listdir(home) if isfile(join(home, f))]
img_all = []
for i in onlyfiles:
    if i[-7:] !='_ru.pdf':
        img_all.append(i)

In [30]:
def determine_image_type (stream_first_4_bytes):
    """Find out the image file type based on the magic number comparison of the first 4 (or 2) bytes"""
    file_type = None
    bytes_as_hex = b2a_hex(stream_first_4_bytes)
    if bytes_as_hex.startswith('ffd8'):
        file_type = '.jpeg'
    elif bytes_as_hex == '89504e47':
        file_type = ',png'
    elif bytes_as_hex == '47494638':
        file_type = '.gif'
    elif bytes_as_hex.startswith('424d'):
        file_type = '.bmp'
    return file_type

def write_file (folder, filename, filedata, flags='w'):
    """Write the file data to the folder and filename combination
    (flags: 'w' for write text, 'wb' for write binary, use 'a' instead of 'w' for append)"""
    result = False
    if os.path.isdir(folder):
        try:
            file_obj = open(os.path.join(folder, filename), flags)
            file_obj.write(filedata)
            file_obj.close()
            result = True
        except IOError:
            pass
    return result


def save_image (lt_image,i):
    page_number = i[:-6]
    images_folder = 'C:/pj/Images2' 
    """Try to save the image data from this LTImage object, and return the file name, if successful"""
    result = None
    if lt_image.stream:
        file_stream = lt_image.stream.get_rawdata()
        file_ext = determine_image_type(file_stream[0:4])
        if file_ext:
            if len(lt_image.name)==4:
                if lt_image.name =='img2':
                    file_name = ''.join([str(page_number), '_1', file_ext])
                    if write_file(images_folder, file_name, lt_image.stream.get_rawdata(), flags='wb'):
                        result = file_name
                if lt_image.name =='img3':
                    file_name = ''.join([str(page_number), '_2', file_ext])
                    if write_file(images_folder, file_name, lt_image.stream.get_rawdata(), flags='wb'):
                        result = file_name
            if lt_image.name =='img4':
                file_name = ''.join([str(page_number), '_1', file_ext])
                if write_file(images_folder, file_name, lt_image.stream.get_rawdata(), flags='wb'):
                    result = file_name
            if lt_image.name =='img5':
                file_name = ''.join([str(page_number), '_2', file_ext])
                if write_file(images_folder, file_name, lt_image.stream.get_rawdata(), flags='wb'):
                    result = file_name
    return result

for i in img_all:
    fp = open(home+i, 'rb')
    parser = PDFParser(fp)
    document = PDFDocument(parser)
    rsrcmgr = PDFResourceManager()
    laparams = LAParams()
    device = PDFPageAggregator(rsrcmgr, laparams=laparams)
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    
    for page in PDFPage.create_pages(document):
        interpreter.process_page(page)
        pdf_item = device.get_result()
        for thing in pdf_item:
            if isinstance(thing, LTImage):
                save_image(thing,i)
            if isinstance(thing, LTFigure):
                find_images_in_thing(thing,i)